# 1. GEE - CAMS PM 2.5 history
Uses the CAMS dataset on Google Earth Engine
and the location of our cities of interest
to compute the mean pollution level at each city,
for different points in time. Also calculates
the historical average for the available period.

In [110]:
import ee
import geemap
import pandas as pd

In [111]:
# Function to add 'date' property (just the day, ignoring hours)
def add_date(image):
    
    date = ee.Date(image.get('model_initialization_datetime')).format('YYYY-MM-dd');
    year = ee.Date(image.get('model_initialization_datetime')).get('year')
    month = ee.Date(image.get('model_initialization_datetime')).get('month')
    week = ee.Date(image.get('model_initialization_datetime')).get('week')
    day = ee.Date(image.get('model_initialization_datetime')).get('day')
    
    forecast_hour = image.get('model_forecast_hour');
    
    
    return image.set({
        "date": date,
        "year": year,
        "month": month,
        "week": week,
        "day": day
    })

In [112]:
def monthly_historical_mean(month):
    
    filtered = cams_collection.filter(ee.Filter.eq('month', month))
    mean = filtered.mean()
    
    return mean.set('month', month)

In [115]:
ee.Initialize()

In [116]:
# All available CAMS images, within a 24h forecast and a 0h UTC
cams_collection = (ee.ImageCollection('ECMWF/CAMS/NRT')
                   .filterDate("2017-01-01", "2023-12-31")
                   .filter(ee.Filter.eq('model_initialization_hour', 0))
                   .filter(ee.Filter.lte('model_forecast_hour', 21))
                   .select('particulate_matter_d_less_than_25_um_surface'))

In [118]:
# Apply the add_date function to each image in the collection
cams_collection = cams_collection.map(add_date)

In [119]:
# First, let's compute the mean for every month and week, at every pixel
unique_months = cams_collection.aggregate_array('month').distinct()
monthly_history = ee.ImageCollection(unique_months.map(monthly_historical_mean))

In [120]:
monthly_size = monthly_history.size().getInfo()
monthly_history = monthly_history.toList(monthly_size)

In [129]:
# Exports each entry in the image to a Drive folder
# Which we will download at once to the output path
def export_monthly_image(image, month):
    
    # Get the image ID (or generate one if the ID is empty)
    # Create export task for the image
    task = ee.batch.Export.image.toAsset(
        image=image,
        description=f'Month_{month}_Mean',
        assetId=f'projects/ee-tree-coverage-cities/assets/pm2p5-monthly-2017-2023/Month_{month}_Mean',  # Replace with your asset path
        region=[-90, -60, 30, 15], 
        scale=44528
    )

    # Start the export task
    task.start()
    print(f'Exporting image for month {month}')

In [130]:
for i, month in enumerate(range(1,13)):
    image = ee.Image(monthly_history.get(i))
    export_monthly_image(image, month)

Exporting image for month 1
Exporting image for month 2
Exporting image for month 3
Exporting image for month 4
Exporting image for month 5
Exporting image for month 6
Exporting image for month 7
Exporting image for month 8
Exporting image for month 9
Exporting image for month 10
Exporting image for month 11
Exporting image for month 12


In [150]:
for i, month in enumerate([5]):
    image = ee.Image(monthly_history.get(i))
    export_monthly_image(image, month)

Exporting image for month 5
